Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')

In [2]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


#### Вывод

Всего в таблице 12 столбцов.
Каждая строка содержит информацию о заёмщике, которая необходима банку для оценки надёжности и платёжеспособности клиентов.

Не понятно почему количество дней найма - отрицательные и что это вообще за значения такие - строка 4 - 340 тысяч дней найма - столько не живут. В столбце есть пропущенные значения. Скорее всего данные из этого столбца не понадобятся для ответа на вопросы исследования.

В столбце, который нам понадобится для ответана вопрос - total_income - пропущенные значения. 

Одинаковое количество пропусков в столбцах total_income и days_employed

Разный регистр в столбце education.

Сначала нужно решить проблему с пропусками. 

### Шаг 2. Предобработка данных

#### Обработка пропусков

In [3]:
#Исключим пропуски
df.isnull().sum() #посмотрим где есть пропуски
df[df['total_income'].isnull()].head() #смотрим тип - это NaN
df[df['days_employed'].isnull()].head()

# убираем отрицательные значения в 'days_employed', чтобы не заменять отрицательной медианой. 
# возможно это измерения не в днях, а в часах. 
df['days_employed'] = df['days_employed'].abs()
df['total_income'] = df['total_income'].abs()

work_days = df['days_employed'].median() #найдём медиану (без экстрима)
df['days_employed'] = df['days_employed'].fillna(work_days) # и заменим медианой пропуски

# попробуем по совету вьера заменить медианами по типу дохода
# получаем таблицу со средними значениями для каждого типа, вспомним про agg()
median_table = df.groupby(['income_type']).agg({'total_income' : 'median'})
print(median_table)

for inc_type in median_table.index:
    df.loc[df['income_type'] == inc_type,'total_income'] = df.loc[df['income_type'] == 
inc_type,'total_income'].fillna(median_table.loc[inc_type, 'total_income'])
#median_table.index - список с индексами в median_table, а индексы - медианы


#income_median = df['total_income'].median()
#df['total_income'] = df['total_income'].fillna(income_median)


df['children'].value_counts() #в нужном для задания столбце children есть значения -1 и 20
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)

 
df['gender'].unique() #получили значение 'XNA' - сейчас это стало так модно)
#(df[df['gender'] == 'XNA'])
df['gender'] = df['gender'].replace('XNA', 'M')


df['dob_years'].unique() #проскачил нолик
age_median = df['dob_years'].median()
df['dob_years'] = df['dob_years'].replace(0, age_median)
#df['dob_years'].unique()
df.isnull().sum() # смотрим - есть ли пропуски

                  total_income
income_type                   
безработный      131339.751676
в декрете         53829.130729
госслужащий      150447.935283
компаньон        172357.950966
пенсионер        118514.486412
предприниматель  499163.144947
сотрудник        142594.396847
студент           98201.625314


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

#### Вывод

В столбцах days_employed и total_income одинаковое количество пропущеных значений - 2174. Возможно люди не в состоянии посчитать сколько они работают или не хотят указывать.

Пропуски в столбце total_income возможны из-за того, что не хотели указывать свой доход или нет таких сведений у людей. Заменил медианой. Оптимально. 

В столбце children встречаются количества детей: -1 и 20 - возможно опечатка. Меняем на 1 и 2. 

В столбце gender - неопределёный пол 'XNA'. заменить можно на любой т.к. Замена не повлияет на результат.

В толбце dob_years - есть нулевые значения. Возможно это женщина и не захотела указывать количество лет. Возможно невнимательность. Заменил медианой.

#### Замена типа данных

In [4]:
df['total_income'] = df['total_income'].astype('int') #перевёл в целочисленные значения столбец total_income, так удобнее считать
df['days_employed'] = df['days_employed'].astype('int') #для удобства переведём в целочисленные значения столбец days_employed
df['education'] = df['education'].str.lower() #приведём к нижнему регистру значения в столбце education для порядка
df['family_status'].value_counts() # Не женат / не замужем - не порядок!
df['family_status'] = df['family_status'].str.lower()
#df['family_status'].value_counts() #порядок
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


#### Вывод

Для ответа на главные вопросы нам понадобятся столбцы purpose, total_income, debt, children.

#### Обработка дубликатов

In [5]:
df.duplicated().sum() #считаем дубликаты, получаем 72.
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum() #дубликатов нет
df.isnull().sum() #пропусков нет

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

#### Вывод

Методами duplicated().sum() - нашли 72 дубликата. Убрал прекрасным методом. Количество дубликатов трудно назвать существенным. Возмжная причина появления дубликатов - невнимательность, человеческий фактор, сбой в записи данных. Возможно один и тот же человек подавал заявку дважды.

#### Лемматизация

In [6]:
text = ' '.join(df['purpose'].unique())
from pymystem3 import Mystem
m = Mystem()
lemmas = m.lemmatize(text)
from collections import Counter
counter = Counter(lemmas)
print(counter) # посмотрим, для чего людям нужен кредит
# функция будет искать леммы в исходных целях кредита
def words(purpose): 
    lemmas = m.lemmatize(purpose)
    try:
        if ('жилье' in lemmas) or ('недвижимость' in lemmas):
            return 'недвижимость'
        if 'автомобиль' in lemmas:
            return 'автомобиль'
        if 'образование' in lemmas:
            return 'образование'
        if 'свадьба' in lemmas:
            return 'свадьба'
        else:
            return 'нет данных'
    except:
        return 'что-то не так'
    
#print(words('образование')) # проверка
df['purpose_lemma'] = df['purpose'].apply(words)
#print(df.head())
df['purpose_lemma'].value_counts() # на какие же цели берут кредиты больше всего?

#df.head(20)

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2323
Name: purpose_lemma, dtype: int64

#### Вывод

Основные цели, которые явно просматриваются среди лемм - недвижимость, автомобиль, образование и свадьба. Самая популярная цель кредита - недвижимость и операции с ней.

#### Категоризация данных

In [7]:
#разделим заёмщиков на родителей и бездетных
children_debt = df[['children', 'debt']].copy() #создадим копию таблицы с необходимыми столбцами
def kids_dept(children): #создадим функцию, которая отделяет родителей от бездетных
    if children < 1:
        return 'нет детей'
    if children >= 1:
        return 'есть дети'
#print(kids_dept(1)) #проверим работу функции
children_debt['groups'] = children_debt['children'].apply(kids_dept) #добавим новый столбец
print(children_debt['groups'].value_counts())
print()

# посмотрим как семейное положение влияет на количество обращений в банк
print(df['family_status'].value_counts())
print()

#создадим функцию, которая разобьёт заёмщиков на категории по уровню дохода
income_debt = df[['total_income', 'debt']].copy()
df['total_income'].max() #максимальное значение 2265604
df['total_income'].min() #минимальное значение 20667
def income_category(total_income):
    if total_income >= 20000 and total_income <= 40000:
        return 'выше бедности'
    if total_income > 40000 and total_income <= 80000:
        return 'средний достаток'
    if total_income > 80000 and total_income <= 100000:
        return 'состоятельные'
    if total_income > 100000:
        return 'богатые'
#income_category(200000) #проверим
#добавим столбец с результатами работы функции
income_debt['category'] = income_debt['total_income'].apply(income_category)
#income_debt.head(10) #посмотрим результат работы функции и метода apply
income_debt_category = income_debt.groupby('category').sum()
income_debt_category_total = income_debt['category'].value_counts()
debt_category = income_debt.groupby('debt')['category'].value_counts()
print(debt_category)
print()
print(income_debt_category_total)
income_debt_category['total'] = income_debt_category_total
#print(income_debt_category)

print()
# понадобятся два столбца для выявления влияния целей кредита на возвращаемость
purpose_dept = df[['purpose_lemma', 'debt']].copy()
purposes_count = df['purpose_lemma'].value_counts() # на какие цели берут кредиты больше всего?
responsible_depts = purpose_dept.groupby('debt')['purpose_lemma'].value_counts() # отделим должников от ответственных
#print(responsible_depts)
print(purposes_count)

print()
education_dict = df[['education', 'education_id']]
#print(len(df['education_id'].unique()))
#print(education_dict.head())
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
print(education_dict.sort_values('education_id'))

print()
family_status_dict = df[['family_status', 'family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
print(family_status_dict.sort_values('family_status_id'))

нет детей    14090
есть дети     7363
Name: groups, dtype: int64

женат / замужем          12339
гражданский брак          4150
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

debt  category        
0     богатые             15603
      состоятельные        2007
      средний достаток     1987
      выше бедности         115
1     богатые              1387
      состоятельные         180
      средний достаток      164
      выше бедности          10
Name: category, dtype: int64

богатые             16990
состоятельные        2187
средний достаток     2151
выше бедности         125
Name: category, dtype: int64

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2323
Name: purpose_lemma, dtype: int64

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное        

#### Вывод

За кредитом чаще обращаются семьи, бездетные люди и заёмщики с высоким доходом. Чаще всего кредиты берут для операции с недвижимостью.

Есть соблазн написать, что чаще всего обращаются за кредитом бездетные семьи с высоким доходом с целью покупки недвижимости, но я бы не стал делать такие выводы на основе данных.

Для категоризации по уровню дохода обратимся к данным Федеральной службы государственной статистики:

-крайняя нищета — доходы ниже прожиточного минимума (до 7-8 тыс.р.)

-нищета — доходы от одного до двух прожиточных минимумов (от 8 до 12 тыс.р.)

-бедность — доходы от 12 до 20 тысяч рублей в месяц.

-выше бедности — доходы от 20 до 30 тысяч рублей в месяц.

-средний достаток — доходы от 30 до 60 тысяч рублей в месяц.

-состоятельные — доходы от 60 до 90 тысяч рублей в месяц.

-богатые — доходы от 90 тысяч рублей в месяц.

Категоризируем согласно этим данным, немного увеличив дипазон, так как данные ФСГС немного преукрашены. Если мы расширим диапозон "выше бедности" от 20т.р. до 40т.р. - это будет больше похоже на рпавду. 

### Шаг 3. Ответы на поставленные вопросы

#### - Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
# посчитаем количество должников
debt_count = children_debt.groupby('groups')['debt'].value_counts() #посчитаем должников и отдающих в срок

debt_count

groups     debt
есть дети  0        6685
           1         678
нет детей  0       13027
           1        1063
Name: debt, dtype: int64

In [9]:
parents_dept = debt_count['есть дети'][1] / 7363 #наверно деления на цифру тоже можно как то избежать, но так проще
child_free_dept = debt_count['нет детей'][1] / 14090

print('Должники среди родителей: {:.1%}'.format(parents_dept)) #выведем в процентах
print('Должники среди бездетных: {:.1%}'.format(child_free_dept))

Должники среди родителей: 9.2%
Должники среди бездетных: 7.5%


#### Вывод

Большее количество должников среди бездетных не означает, что им не чтоит выдавать кредиты, они чаще обращаются за займом. Если смотреть на процент, то должников среди бездетных заёмщиков меньше. Разницу в 1.7% трудно назвать показательной, скорее она незначительная.

#### - Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [10]:
# создадим таблицу с необходимыми столбцами
family_status_debt = df[['family_status', 'family_status_id', 'debt']].copy()
family_status_debt.head()
family_status_debt['family_status'].value_counts()

pd.DataFrame(family_status_debt.groupby('debt')['family_status'].value_counts())

# на основе полученных данных сделаем общуу таблицу
data = [['женат / замужем', 931, 12339],
       ['гражданский брак', 388, 4150],
        ['не женат / не замужем', 274, 2536],
        ['в разводе', 85, 1110],
        ['вдовец / вдова', 63, 896]]
columns = ['семейное положение', 'должники', 'общее кол-во']
total_debt = pd.DataFrame(data = data, columns = columns)
index = total_debt['должники'] / total_debt['общее кол-во'] # найдём отношение
def index_per(index):
    return '{:.0%}'.format(index)
total_debt['процент должников'] = index.apply(index_per) # добавим столбец с процентным соотношением
total_debt


,семейное положение,должники,общее кол-во,процент должников
0,женат / замужем,931,12339,8%
1,гражданский брак,388,4150,9%
2,не женат / не замужем,274,2536,11%
3,в разводе,85,1110,8%
4,вдовец / вдова,63,896,7%


#### Вывод

Должники среди тех кто не женат / не замужем - лидируют - 11%, хотя больше всего задолжали те, кто женат / замужем - 931 чел., но они и чаще обращаются в банк - 12339 чел.

#### - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [11]:
# полученные при категоризации данные свдём в таблицу
data = [['богатые', 1387, 16990],
       ['выше бедности', 10, 125],
       ['состоятельные', 180, 2187],
       ['средний достаток', 164, 2151]]
columns = ['категория', 'должники', 'общее кол-во']
income_table = pd.DataFrame(data = data, columns = columns)
percent = income_table['должники'] / income_table['общее кол-во']



#переведём столбец 'процент задолжавших' в проценты
def index_perc(percent):
    return '{:.1%}'.format(percent)
income_table['процент должников'] = percent.apply(index_perc)
income_table.sort_values(by='процент должников', ascending = False)


,категория,должники,общее кол-во,процент должников
0,богатые,1387,16990,8.2%
2,состоятельные,180,2187,8.2%
1,выше бедности,10,125,8.0%
3,средний достаток,164,2151,7.6%


#### Вывод

Богатые и состоятельные люди не спешат отдавать кредиты. В целом разница в процентах совсем мала, что бы сделать вывод о влиянии финансового положения на возвращаемость кредитов.

#### - Как разные цели кредита влияют на его возврат в срок?

In [12]:
#можно так:

#создадим табличку
data = [['недвижимость', 782, 10811], 
        ['автомобиль', 403, 4306], 
       ['образование', 370, 4013],
       ['свадьба', 186, 2323]]
columns = ['цель', 'должники', 'количество заёмщиков']
dept_and_purp = pd.DataFrame(data=data, columns=columns)

percent_dept = dept_and_purp['должники'] / dept_and_purp['количество заёмщиков']

#переведём столбец 'процент задолжавших' в проценты
def index_perc_1(percent_dept):
    return '{:.1%}'.format(percent_dept)

dept_and_purp['процент задолжавших'] = percent_dept.apply(index_perc_1)
dept_and_purp.sort_values(by='процент задолжавших', ascending = False)

# а можно так
# используем метод pivot_table()

df_pivot = df.pivot_table(index=['purpose_lemma'], columns= 'debt', values='purpose', aggfunc='count')
df_pivot_with_reset_index = df_pivot.reset_index()

df_pivot_with_reset_index['общее количество'] = df_pivot_with_reset_index[0] + df_pivot_with_reset_index[1]
percent_depted = df_pivot_with_reset_index[1] / df_pivot_with_reset_index['общее количество']

def index_perc_2(percent_depted):
    return '{:.1%}'.format(percent_depted)

df_pivot_with_reset_index['процент задолжавших'] = percent_depted.apply(index_perc_2)
df_pivot_with_reset_index.sort_values(by='процент задолжавших', ascending = False)

debt,purpose_lemma,0,1,общее количество,процент задолжавших
0,автомобиль,3903,403,4306,9.4%
2,образование,3643,370,4013,9.2%
3,свадьба,2137,186,2323,8.0%
1,недвижимость,10029,782,10811,7.2%


#### Вывод

Цель "недвижимость" уверенно лидирует по количеству заявок. Однако те, кто берёт кредит с целью "автомобиль" - не хотят возвращать деньги вовремя. Да - сейчас машина не решает прблемы, а только создаёт их.

### Шаг 4. Общий вывод

Есть ли зависимость между наличием детей и возвратом кредита в срок?
Должников с детьми больше - 9,2% против 7,5%. Разница не велика, чтобы однозначно увидеть зависимость между наличием детей и возвратом кредита в срок. Однако расхождение говорит о том, что родители в определённых ситуациях могут предпочесть потратить деньги на детей, нежели платить за кредит в срок.

Есть ли зависимость между семейным положением и возвратом кредита в срок?
Те, кто не женат / не замужем - 11% или состоят в гражданском браке - 9% не спешат идти в банк в день погашения очередного платежа по кредиту. Семейные люди - 8% куда чаще обращаются в кредитный отдел, но и ответственных семей больше. Семья дело такое! Сейчас принято считать, что гражданский брак и женат / замужем - это одно и тоже. Тем не менее состоящие в гражданском браке платят по кредиту чаще не в срок. Те, кто имел семейный опыт - "в разводе" - 8%, "вдовцы" - 7% и "женат / замужем" относятся к возврату кредита более ответственно, чем те кому предстоит этот опыт познать - "не женат / не замужем" и "гражданский брак". 

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Обеспеченные люди чаще остальных обращаются за кредитом. Интересное наблюдение, может стоит перенять этот опыт? Однако люди с высоким доходом чаще пропускают день выплаты по кредиту. Процент должников среди заёмщиков со средним достатком ниже. Разница между категориями не велика. Напрашивается вывод, что уровень дохода не сильно влияет на возврат кредита в срок.

Как разные цели кредита влияют на его возврат в срок?
Те, у кого нет автомобиля - мечтают его купить, а те, у кого автомобиль есть - мечтают его продать. Машина в России - это куча проблем, а не удовольствие от владения. Видимо поэтому те, кто берут кредит на автомобиль - не спешат его возвращать. С образованием похожая ситуация. Интересно, что кредиты на недвижимость берут намного чаще, а возвращают в срок регулярнее. Вероятно ответственность перед банком выше.